# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
D. Mortimer  ->  D. Mortimer  |  ['D. Mortimer']
B. Setterholm  ->  B. Setterholm  |  ['B. Setterholm']
F. Walter  ->  F. Walter  |  ['F. Walter']
A. Kospal  ->  A. Kospal  |  ['A. Kospal']
T. Henning  ->  T. Henning  |  ['T. Henning']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']
Arxiv has 51 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.03982


extracting tarball to tmp_2408.03982...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.04038


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2690/1211882699.py:51: LatexWarning: 2408.03982 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.04475


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2690/1211882699.py:51: LatexWarning: 2408.04465 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2408.04475...

 done.


Found 134 bibliographic references in tmp_2408.04475/main.bbl.
Error retrieving bib data for marley21: 'author'
Error retrieving bib data for grieves21: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.04038-b31b1b.svg)](https://arxiv.org/abs/2408.04038) | **Recent and Upcoming Upgrades for MIRC-X and MYSTIC on the CHARA Array**  |
|| N. Ibrahim, et al. -- incl., <mark>D. Mortimer</mark>, <mark>B. Setterholm</mark> |
|*Appeared on*| *2024-08-09*|
|*Comments*| *11 pages, 9 figures, Submitted to SPIE in Yokohama 2024*|
|**Abstract**|            MIRC-X and MYSTIC are six-telescope near-infrared beam (1.08-2.38 ${\mu}$m) combiners at the CHARA Array on Mt Wilson CA, USA. Ever since the commissioning of MIRC-X (J and H bands) in 2018 and MYSTIC (K bands) in 2021, they have been the most popular and over-subscribed instruments at the array. Observers have been able to image stellar objects with sensitivity down to 8.1 mag in H and 7.8 mag in K-band under the very best conditions. In 2022 MYSTIC was upgraded with a new ABCD mode using the VLTI/GRAVITY 4-beam integrated optics chip, with the goal of improving the sensitivity and calibration. The ABCD mode has been used to observe more than 20 T Tauri stars; however, the data pipeline is still being developed. Alongside software upgrades, we detail planned upgrades to both instruments in this paper. The main upgrades are: 1) Adding a motorized filter wheel to MIRC-X along with new high spectral resolution modes 2) Updating MIRC-X optics to allow for simultaneous 6T J+H observations 3) Removing the warm window between the spectrograph and the warm optics in MYSTIC 4) Adding a 6T ABCD mode to MIRC-X in collaboration with CHARA/SPICA 5) Updating the MIRC-X CRED-ONE camera funded by Prof. Kraus from U. Exeter 6) Carrying out science verification of the MIRC-X polarization mode 7) Developing new software for ABCD-mode data reduction and more efficient calibration routines. We expect these upgrades to not only improve the observing experience, but also increase the sensitivity by 0.4 mag in J+H-bands, and 1 mag in K-band.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.04135-b31b1b.svg)](https://arxiv.org/abs/2408.04135) | **JWST Observations of Starbursts: Massive Star Clusters in the Central Starburst of M82**  |
|| R. C. Levy, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-08-09*|
|*Comments*| *Resubmitted to ApJL*|
|**Abstract**|            We present a near infrared (NIR) candidate star cluster catalog for the central kiloparsec of M82 based on new JWST NIRCam images. We identify star cluster candidates using the F250M filter, finding 1357 star cluster candidates with stellar masses $>10^4$ M$_\odot$. Compared to previous optical catalogs, nearly all (87%) of the candidates we identify are new. The star cluster candidates have a median intrinsic cluster radius of $\approx$1 pc and have stellar masses up to $10^6$ M$_\odot$. By comparing the color-color diagram to dust-free yggdrasil stellar population models, we estimate that the star cluster candidates have A$_{\rm V}\sim3-24$ mag, corresponding to A$_{\rm 2.5\mu m}\sim0.3-2.1$ mag. There is still appreciable dust extinction towards these clusters into the NIR. We measure the stellar masses of the star cluster candidates, assuming ages of 0 and 8 Myr. The slope of the resulting cluster mass function is $\beta=1.9\pm0.2$, in excellent agreement with studies of star clusters in other galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.04475-b31b1b.svg)](https://arxiv.org/abs/2408.04475) | **TOI-2490b- The most eccentric brown dwarf transiting in the brown dwarf desert**  |
|| B. A. Henderson, et al. -- incl., <mark>T. Henning</mark>, <mark>T. Trifonov</mark> |
|*Appeared on*| *2024-08-09*|
|*Comments*| *Accepted for publication in MNRAS, 18 pages, 14 figures*|
|**Abstract**|            We report the discovery of the most eccentric transiting brown dwarf in the brown dwarf desert, TOI02490b. The brown dwarf desert is the lack of brown dwarfs around main sequence stars within $\sim3$~AU and is thought to be caused by differences in formation mechanisms between a star and planet. To date, only $\sim40$ transiting brown dwarfs have been confirmed. \systemt is a $73.6\pm2.4$ \mjupnospace, $1.00\pm0.02$ \rjup brown dwarf orbiting a $1.004_{-0.022}^{+0.031}$ \msunnospace, $1.105_{-0.012}^{+0.012}$ \rsun sun-like star on a 60.33~d orbit with an eccentricity of $0.77989\pm0.00049$. The discovery was detected within \tess sectors 5 (30 minute cadence) and 32 (2 minute and 20 second cadence). It was then confirmed with 31 radial velocity measurements with \feros by the WINE collaboration and photometric observations with the Next Generation Transit Survey. Stellar modelling of the host star estimates an age of $\sim8$~Gyr, which is supported by estimations from kinematics likely placing the object within the thin disc. However, this is not consistent with model brown dwarf isochrones for the system age suggesting an inflated radius. Only one other transiting brown dwarf with an eccentricity higher than 0.6 is currently known in the brown dwarf desert. Demographic studies of brown dwarfs have suggested such high eccentricity is indicative of stellar formation mechanisms.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.03982-b31b1b.svg)](https://arxiv.org/abs/2408.03982) | **JWST/NIRSpec WIDE survey: a z=4.6 low-mass star-forming galaxy hosting a jet-driven shock with low ionisation and solar metallicity**  |
|| F. D'Eugenio, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-08-09*|
|*Comments*| *20 pages, 10 figures, 4 tables. Submitted to MNRAS, comments welcome*|
|**Abstract**|            We present NIRSpec/MSA observations from the JWST large-area survey WIDE, targeting the rest-frame UV-optical spectrum of Ulema, a radio-AGN host at redshift z=4.6348. The low-resolution prism spectrum displays high equivalent width nebular emission, with remarkably high ratios of low-ionisation species of oxygen, nitrogen and sulphur, relative to hydrogen; auroral O$^+$ emission is clearly detected, possibly also C$^+$. From the high-resolution grating spectrum, we measure a gas velocity dispersion $\sigma$~400 km s$^{-1}$, broad enough to rule out star-forming gas in equilibrium in the gravitational potential of the galaxy. Emission-line ratio diagnostics suggest that the nebular emission is due to a shock which ran out of pre-shock gas. To infer the physical properties of the system, we model simultaneously the galaxy spectral energy distribution (SED) and shock-driven line emission under a Bayesian framework. We find a relatively low-mass, star-forming system (M* = 1.4$\times$10^{10} M$_\odot$, SFR = 70 M$_\odot$ yr$^{-1}$), where shock-driven emission contributes 50 per cent to the total H$\beta$ luminosity. The nebular metallicity is near solar - three times higher than that predicted by the mass-metallicity relation at z=4.6, possibly related to fast-paced chemical evolution near the galaxy nucleus. We find no evidence for a recent decline in the SFR of the galaxy, meaning that, already at this early epoch, fast radio-mode AGN feedback was poorly coupled with the bulk of the star-forming gas; therefore, most of the feedback energy must end up in the galaxy halo, setting the stage for future quenching.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.04465-b31b1b.svg)](https://arxiv.org/abs/2408.04465) | **A Multi-wavelength, Multi-epoch Monitoring Campaign of Accretion Variability in T Tauri Stars from the ODYSSEUS Survey. III. Optical Spectra**  |
|| J. Wendeborn, et al. -- incl., <mark>A. Kospal</mark> |
|*Appeared on*| *2024-08-09*|
|*Comments*| *39 pages, 10 figures, 12 tables*|
|**Abstract**|            Classical T Tauri Stars (CTTSs) are highly variable stars that possess gas- and dust-rich disks from which planets form. Much of their variability is driven by mass accretion from the surrounding disk, a process that is still not entirely understood. A multi-epoch optical spectral monitoring campaign of four CTTSs (TW Hya, RU Lup, BP Tau, and GM Aur) was conducted along with contemporaneous HST UV spectra and ground-based photometry in an effort to determine accretion characteristics and gauge variability in this sample. Using an accretion flow model, we find that the magnetospheric truncation radius varies between 2.5-5 R* across all of our observations. There is also significant variability in all emission lines studied, particularly Halpha, Hbeta, and Hgamma. Using previously established relationships between line luminosity and accretion, we find that, on average, most lines reproduce accretion rates consistent with accretion shock modeling of HST spectra to within 0.5 dex. Looking at individual contemporaneous observations, however, these relationships are less accurate, suggesting that variability trends differ from the trends of the population and that these empirical relationships should be used with caution in studies of variability.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.04038/./translation.jpg', 'tmp_2408.04038/./o.png', 'tmp_2408.04038/./spicaFT.png']
copying  tmp_2408.04038/./translation.jpg to _build/html/
copying  tmp_2408.04038/./o.png to _build/html/
copying  tmp_2408.04038/./spicaFT.png to _build/html/
exported in  _build/html/2408.04038.md
    + _build/html/tmp_2408.04038/./translation.jpg
    + _build/html/tmp_2408.04038/./o.png
    + _build/html/tmp_2408.04038/./spicaFT.png
found figures ['tmp_2408.04135/./F140M-F250M-F360M_asinh_inset_HST.png', 'tmp_2408.04135/./NIRCam_F250M_AgeComp_cCMF_Fraction_ClusterComp_PowerLawFit.png', 'tmp_2408.04135/./F250M-F360M_F140M-F250M_SSP_cmKDE.png']
copying  tmp_2408.04135/./F140M-F250M-F360M_asinh_inset_HST.png to _build/html/
copying  tmp_2408.04135/./NIRCam_F250M_AgeComp_cCMF_Fraction_ClusterComp_PowerLawFit.png to _build/html/
copying  tmp_2408.04135/./F250M-F360M_F140M-F250M_SSP_cmKDE.png to _build/html/
exported in  _build/html/2408.04135.md
    + _build/html/tmp_2408.04135/./

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\baselinestretch}{1.0}$</div>



<div id="title">

# Recent and Upcoming Upgrades for MIRC-X and MYSTIC on the CHARA Array

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.04038-b31b1b.svg)](https://arxiv.org/abs/2408.04038)<mark>Appeared on: 2024-08-09</mark> -  _11 pages, 9 figures, Submitted to SPIE in Yokohama 2024_

</div>
<div id="authors">

N. Ibrahim, et al. -- incl., <mark>D. Mortimer</mark>, <mark>B. Setterholm</mark>

</div>
<div id="abstract">

**Abstract:** MIRC-X and MYSTIC are six-telescope near-infrared beam (1.08-2.38 $\mu$ m) combiners at the CHARA Array on Mt Wilson CA, USA. Ever since the commissioning of MIRC-X (J and H bands) in 2018 and MYSTIC (K bands) in 2021, they have been the most popular and over-subscribed instruments at the array. Observers have been able to image stellar objects with sensitivity down to 8.1 mag in H and 7.8 mag in K-band under the very best conditions. In 2022 MYSTIC was upgraded with a new ABCD mode using the VLTI/GRAVITY 4-beam integrated optics chip, with the goal of improving the sensitivity and calibration. The ABCD mode has been used to observe more than 20 T Tauri stars; however, the data pipeline is still being developed. Alongside software upgrades, we detail planned upgrades to both instruments in this paper. The main upgrades are: 1) Adding a motorized filter wheel to MIRC-X along with new high spectral resolution modes 2) Updating MIRC-X optics to allow for simultaneous 6T J+H observations 3) Removing the warm window between the spectrograph and the warm optics in MYSTIC 4) Adding a 6T ABCD mode to MIRC-X in collaboration with CHARA/SPICA 5) Updating the MIRC-X CRED-ONE camera funded by Prof. Kraus from U. Exeter 6) Carrying out science verification of the MIRC-X polarization mode 7) Developing new software for ABCD-mode data reduction and more efficient calibration routines. We expect these upgrades to not only improve the observing experience, but also increase the sensitivity by 0.4 mag in J+H-bands, and 1 mag in K-band.

</div>

<div id="div_fig1">

<img src="tmp_2408.04038/./translation.jpg" alt="Fig5" width="100%"/>

**Figure 5. -** (left) The filter wheel is assembled and is being tested at UM lab. Special grating holders were 3D printed and can be seen in the blue. A translation stage (circled in green) will hold the different lenses and move desired on into the optical axis so observers can switch between modes (Right) CAD of the filter wheel, the grating holder, and the moving lens system.
 (*fig:wheel*)

</div>
<div id="div_fig2">

<img src="tmp_2408.04038/./o.png" alt="Fig1" width="100%"/>

**Figure 1. -** Example of MYSTIC ABCD observations as seen on the detector. Four beams are combined and output as 24 parallel lines, sampling four phases of the visibility contrast at each wavelength channel for 6 baselinesmystic (*fig:abcd*)

</div>
<div id="div_fig3">

<img src="tmp_2408.04038/./spicaFT.png" alt="Fig2" width="100%"/>

**Figure 2. -** CAD view of the SPICA-FT (in pink) positioned near the MIRC-X spectrograph spicaft (*fig:SPICAFT*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.04038"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\starbug}{\textsc{starbug ii}}$
$\newcommand{\cigale}{\textsc{cigale}}$
$\newcommand{\yggdrasil}{\textsc{yggdrasil}}$
$\newcommand{\MJysr}{MJy~sr^{-1}}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\lsun}{L_\odot}$
$\newcommand{\sourcextractor}{{\tt SourceXtractor++}}$
$\newcommand{\ml}{\Upsilon_*}$
$\newcommand{\mstar}{M_*}$
$\newcommand{\per}{^{-1}}$
$\newcommand{\citetJD}{M.~J. Jiménez-Donaire et al. (in prep.)}$
$\newcommand{\citepJD}{M.~J. Jiménez-Donaire et al. in prep.}$
$\newcommand{\change}$
$\newcommand{\todo}{\textcolor{WildStrawberry}}$
$\newcommand{ÅPF}{\altaffiliation{NSF Astronomy and Astrophysics Postdoctoral Fellow}}$
$\newcommand{\Arizona}{\affiliation{Steward Observatory, University of Arizona, Tucson, AZ 85721, USA}}$
$\newcommand{\Maryland}{\affiliation{Department of Astronomy, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{\JSI}{\affiliation{Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{\INA}{\affiliation{Instituto Nacional de Astrofísica, Óptica y Electrónica, Luis Enrique Erro 1, Tonantzintla, 72840 Puebla, Mexico}}$
$\newcommand{\BUAP}{\affiliation{Benemérita Universidad Autónoma de Puebla, Av. San Manuel, 72000 Puebla, Mexico}}$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\MPIA}{\affiliation{Max Planck Institut for Astronomy, Konigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\KU}{\affiliation{Department of Physics and Astronomy, University of Kansas, 1251 Wescoe Hall Drive, Lawrence, KS 66045, USA}}$
$\newcommand{\UWyoming}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\Leiden}{\affiliation{Leiden Observatory, Leiden University, P.O.~Box 9513, 2300~RA~Leiden, The Netherlands}}$
$\newcommand{\UGent}{\affiliation{Sterrenkundig Observatorium, Ghent University, Krijgslaan 281 - S9, B-9000 Gent, Belgium}}$
$\newcommand{\France}{\affiliation{Collège de France, 11 Pl. Marcelin Berthelot, 75231 Paris, France}}$
$\newcommand{\ParisObs}{\affiliation{Observatoire de Paris, 61 avenue de l’Observatoire, 75014 Paris, France}}$
$\newcommand{\ESOST}{\affiliation{European Space Agency, c/o STScI, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\ITA}{\affiliation{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str. 2, D-69120 Heidelberg, Germany}}$
$\newcommand{\IWR}{\affiliation{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\SOFIA}{\affiliation{Stratospheric Observatory for Infrared Astronomy, NASA Ames Research Center, Mail Stop 204-14, Moffett Field, CA 94035, USA}}$
$\newcommand{\JPL}{\affiliation{Jet Propulsion Laboratory, California Institute of Technology, 4800 Oak Grove Dr., Pasadena, CA 91109, USA}}$
$\newcommand{\UdeC}{\affiliation{Departamento de Astronomía, Universidad de Concepción, Barrio Universitario, Concepción, Chile}}$
$\newcommand{\NMMT}{\affiliation{New Mexico Institute of Mining and Technology, 801 Leroy Place, Socorro, NM 87801, USA}}$
$\newcommand{\NRAOSocorro}{\affiliation{National Radio Astronomy Observatory, P.O. Box O, 1003 Lopezville Road, Socorro, NM 87801, USA}}$
$\newcommand{\OSU}{\affiliation{Department of Astronomy, The Ohio State University, Columbus, OH 43210, USA}}$
$\newcommand{\UGR}{\affiliation{Dept. Física Teórica y del Cosmos, Universidad de Granada, 18071, Granada, Spain }}$
$\newcommand{\OAN}{\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}}$
$\newcommand{\YS}{\affiliation{Centro de Desarrollos Tecnológicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain}}$
$\newcommand{\Swinburne}{\affiliation{Centre for Astrophysics and Supercomputing, Swinburne University of Technology, Hawthorn, VIC 3122, Australia}}$
$\newcommand{\ASTROTD}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D)}}$
$\newcommand$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# JWST Observations of Starbursts: Massive Star Clusters in the Central Starburst of M 82

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.04135-b31b1b.svg)](https://arxiv.org/abs/2408.04135)<mark>Appeared on: 2024-08-09</mark> -  _Resubmitted to ApJL_

</div>
<div id="authors">

R. C. Levy, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a near infrared (NIR) candidate star cluster catalog for the central kiloparsec of M 82 based on new JWST NIRCam images. We identify star cluster candidates using the F250M filter, finding 1357 star cluster candidates with stellar masses $>10^4$ M $_\odot$ . Compared to previous optical catalogs, nearly all (87 \% ) of the candidates we identify are new. The star cluster candidates have a median intrinsic cluster radius of $\approx$ 1 pc and have stellar masses up to $10^6$ M $_\odot$ . By comparing the color-color diagram to dust-free yggdrasil stellar population models, we estimate that the star cluster candidates have A $_{\rm V}\sim3-24$ mag, corresponding to A $_{\rm 2.5\mu m}\sim0.3-2.1$ mag. There is still appreciable dust extinction towards these clusters into the NIR. We measure the stellar masses of the star cluster candidates, assuming ages of 0 and 8 Myr. The slope of the resulting cluster mass function is $\beta=1.9\pm0.2$ , in excellent agreement with studies of star clusters in other galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2408.04135/./F140M-F250M-F360M_asinh_inset_HST.png" alt="Fig4" width="100%"/>

**Figure 4. -** The central 870 pc of M 82 seen with NIRCam showing F140M (blue), F250M (green), and F360M (red) on an asinh scale. $\change${Insets show zoom-ins to 50 pc $\times$ 50 pc regions around some representative massive star cluster candidates.} The circles indicate the massive clusters identified in the NIRCam image, where the radius of the circle is the fitted (deconvolved) radius (see Section \ref{sec:prop} and Table \ref{tab:catalog}). Blue (pink) circles show clusters which do (do not) overlap with the HST catalog \citep[][see Section \ref{ssec:comp_other_cats}]{Mayya2008}. Clusters 1 and 2 are labeled (see Sections \ref{ssec:radii} and \ref{ssec:comp_other_cats}). The right insets show HST three-color images at the same locations (blue: F435W, green: F555W, red: F814W). The light blue circles in the HST insets show the star clusters identified by \citet{Mayya2008} in these selected regions using the same HST data, where the radius of each circle is 5 pc. The pink and blue circles are the same as in the left panels. (*fig:multicolor*)

</div>
<div id="div_fig2">

<img src="tmp_2408.04135/./NIRCam_F250M_AgeComp_cCMF_Fraction_ClusterComp_PowerLawFit.png" alt="Fig5" width="100%"/>

**Figure 5. -** The cumulative CMFs shown as a fraction of the number of clusters for different assumed ages (colored solid curves). The blue and orange shaded regions on the 0 Myr and 8 Myr curves indicates the approximate range of mass increase due to extinction $\change${which scales with M$_*$ (dark) or which is constant with M$_*$ (light); see Section \ref{sssec:dust} for details.} The gray shaded region shows masses below $10^4$ $\msun$ which are not included in the final catalog. The purple line and shaded region shows the power-law fit for the 0 Myr CMF for M$_*>10^4$ $\msun$. The dashed orange curve shows the cumulative CMF for the nuclear star clusters identified by \citet{Mayya2008} which have an assumed age of 8 Myr. (*fig:cmf*)

</div>
<div id="div_fig3">

<img src="tmp_2408.04135/./F250M-F360M_F140M-F250M_SSP_cmKDE.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\change${A color-color plot of the clusters (black dots). The contours show the density of the points; contour levels show 10, 25, 50, and 75\% density levels for clusters which overlap with the \citet{Mayya2008} catalog (blue) and those only detected with NIRCam (pink). The colored tracks show $\yggdrasil$ SSP models assuming no ($\rm{f_{cov}}=0$; blue) and maximal ($\rm{f_{cov}}=1$; red) contributions from nebular emission. The green arrow shows the slope of the reddening vector with A$_{\rm V}=3-24$ mag marked in increments of 3. The clusters in M 82 detected with NIRCam show significant reddening with respect to the dust-free SSP tracks.} (*fig:color-color*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.04135"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mjup}{ M_{\rm J}\space}$
$\newcommand{\mjupnospace}{ M_{\rm J}}$
$\newcommand{\rjup}{ R_{\rm J}\space}$
$\newcommand{\rjupnospace}{ R_{\rm J}}$
$\newcommand{\msun}{ M_{\odot}\space}$
$\newcommand{\msunnospace}{ M_{\odot}}$
$\newcommand{\rsun}{ R_{\odot}\space}$
$\newcommand{\rsunnospace}{ R_{\odot}}$
$\newcommand{\lsun}{ L_{\odot}\space}$
$\newcommand{\tess}{{\it TESS\space}}$
$\newcommand{\tessnospace}{{\it TESS}}$
$\newcommand{\gaia}{{\it Gaia\space}}$
$\newcommand{\gaianospace}{{\it Gaia}}$
$\newcommand{\ngts}{{NGTS\space}}$
$\newcommand{\ngtsnospace}{{NGTS}}$
$\newcommand{\feros}{{FEROS\space}}$
$\newcommand{\ferosnospace}{{FEROS}}$
$\newcommand{\LSO}{La Silla Observatory\space}$
$\newcommand{\PAR}{Paranal Observatory\space}$
$\newcommand{\teff}{{T_{\rm eff}\space}}$
$\newcommand{\teffnospace}{{T_{\rm eff}}}$
$\newcommand{\logg}{{\log g\space}}$
$\newcommand{\loggnospace}{{\log g}}$
$\newcommand{\vsini}{{v\sin{i}\space}}$
$\newcommand{\vsininospace}{{v\sin{i}}}$
$\newcommand{\msini}{{M\sin{i}\space}}$
$\newcommand{\msininospace}{{M\sin{i}}}$
$\newcommand{\feh}{[Fe/H]\space}$
$\newcommand{\fehnospace}{[Fe/H]}$
$\newcommand{\systemt}{{\rm TOI-2490b\space}}$
$\newcommand{\systemtnospace}{{\rm TOI-2490b}}$
$\newcommand{\systemA}{{\rm TOI-2490\space}}$
$\newcommand{\systemAnospace}{{\rm TOI-2490}}$
$\newcommand{\fsed}{f_\mathrm{sed}\space}$
$\newcommand{\fsednospace}{f_\mathrm{sed}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# TOI-2490b- The most eccentric brown dwarf transiting in the brown dwarf desert

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.04475-b31b1b.svg)](https://arxiv.org/abs/2408.04475)<mark>Appeared on: 2024-08-09</mark> -  _Accepted for publication in MNRAS, 18 pages, 14 figures_

</div>
<div id="authors">

B. A. Henderson, et al. -- incl., <mark>T. Henning</mark>, <mark>T. Trifonov</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of the most eccentric transiting brown dwarf  in the brown dwarf desert, $\systemtnospace$ . The brown dwarf desert is the lack of brown dwarfs around main sequence stars within $\sim3$ AU and is thought to be caused by differences in formation mechanisms between a star and planet. To date, only $\sim40$ transiting brown dwarfs have been confirmed. $\systemt$ is a $73.6\pm2.4$ $\mjupnospace$ , $1.00\pm0.02$ $\rjup$ brown dwarf orbiting a $1.004_{-0.022}^{+0.031}$ $\msunnospace$ , $1.105_{-0.012}^{+0.012}$ $\rsun$ sun-like star on a 60.33 d orbit with an eccentricity of $0.77989\pm0.00049$ . The discovery was detected within $\tess$ sectors 5 (30 minute cadence) and 32 (2 minute and 20 second cadence). It was then confirmed with 31 radial velocity measurements with $\feros$ by the WINE collaboration and  photometric observations with the Next Generation Transit Survey. Stellar modelling of the host star estimates an age of $\sim8$ Gyr, which is supported by estimations from kinematics likely placing the object within the thin disc. However, this is not consistent with model brown dwarf isochrones for the system age suggesting an inflated radius. Only one other transiting brown dwarf with an eccentricity higher than 0.6 is currently known in the brown dwarf desert. Demographic studies of brown dwarfs have suggested such high eccentricity is indicative of stellar formation mechanisms.

</div>

<div id="div_fig1">

<img src="tmp_2408.04475/./Mass_Radius_both_isochrones.png" alt="Fig12" width="100%"/>

**Figure 12. -** The mass-radius plot for all known transiting brown dwarfs and low mass stars from 12 to 150 $\mjup$nospace, excluding RIK-72b  ([David, et. al 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...872..161D))  and 2MASS J05352184–0546085 from [Stassun, Mathieu and Valenti (2006)](https://ui.adsabs.harvard.edu/abs/2006Natur.440..311S) due to their youth and therefore large radii. Grey circles are brown dwarfs, whereas grey stars are low mass stars. The dashed vertical lines highlight the 13 and 80 $\mjup$ borders for the brown dwarf regime. $\systemt$ is the black triangle. Model isochrones are also plotted: solid lines are the [Baraffe, et. al (2003)](https://ui.adsabs.harvard.edu/abs/2003A&A...402..701B), [Baraffe, et. al (2015)](https://ui.adsabs.harvard.edu/abs/2015A&A...577A..42B) iscochrones for 0.5 (pink), 1 (light blue), 5 (green) and 10 (dark mauve) Gyr. The breaks in these isochrones are where we switch to the updated [Baraffe, et. al (2015)](https://ui.adsabs.harvard.edu/abs/2015A&A...577A..42B) iscochrones from [Baraffe, et. al (2003)](https://ui.adsabs.harvard.edu/abs/2003A&A...402..701B). The dashed and dotted isochrones show the marley21 Sonora Bobcat models for $\feh$ of 0.0 dex and +0.5 dex respectively. These are plotted for 0.4 (dark blue), 1 (light blue), 4 (bright purple) and 10 (dark mauve) Gyr. All objects can be seen in Table \ref{appendix:tables} along with their sources. This table has been updated from grieves21 and [ and Henderson (2024)](https://ui.adsabs.harvard.edu/abs/2024MNRAS.tmp..603H).  (*fig:massrad*)

</div>
<div id="div_fig2">

<img src="tmp_2408.04475/./PT_all.png" alt="Fig13" width="100%"/>

**Figure 13. -** Models generated using egp+ showing how the temperature of the atmosphere of $\systemt$ is predicted to change across 3 orbits at different pressure levels. Panels 2, 3 and 4 show the different $f_{sed}$ values tested. The first panel shows the cloud free model of HD 80606b ([Mayorga, et. al (2021)](https://ui.adsabs.harvard.edu/abs/2021ApJ...915...41M)) for comparison. (*fig:pt_hd*)

</div>
<div id="div_fig3">

<img src="tmp_2408.04475/./77437543_DSS.png" alt="Fig1" width="100%"/>

**Figure 1. -** DSS image showing $\systemA$(green) with other nearby objects. North and East are shown on the figure in the top left corner as well as a 0.5 arcmin position angle (PA) arrow for scale. (*fig:dss*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.04475"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

156  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
